<a href="https://colab.research.google.com/github/yyduyuxuan/Machine-Learning-for-Data-Driven-Inventory-Replenishment-Evidence-from-the-M5-Retail-Dataset/blob/main/Baseline_(R%2CS)_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import pandas as pd
import numpy as np
from glob import glob
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
S_base = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/S_base.csv")

Mounted at /content/drive


In [ ]:
S_base.tail()

,Unnamed: 0,unique_id,model,ds,mu_hat_H,SS_baseline,S_baseline,y_true_H
229591,229591,HOUSEHOLD_2_371_WI_3,Theta,2016-01-22,2.071476,2.216932,5,0.0
229592,229592,HOUSEHOLD_2_371_WI_3,Theta,2016-01-23,2.071165,2.216932,5,0.0
229593,229593,HOUSEHOLD_2_371_WI_3,Theta,2016-01-24,2.070855,2.216932,5,0.0
229594,229594,HOUSEHOLD_2_371_WI_3,Theta,2016-01-25,2.070544,2.216932,5,0.0
229595,229595,HOUSEHOLD_2_371_WI_3,Theta,2016-01-26,1.710047,2.216932,4,2.0


In [ ]:
demand_1y = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Supervised Project/demand_1y.csv")

In [ ]:
demand_1y.tail()

,Unnamed: 0,unique_id,date,sales,item_id,store_id,dept_id,cat_id,state_id,d
58030,58030,HOUSEHOLD_2_371_WI_3,2016-01-24,0,HOUSEHOLD_2_371,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,1822
58031,58031,HOUSEHOLD_2_371_WI_3,2016-01-25,0,HOUSEHOLD_2_371,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,1823
58032,58032,HOUSEHOLD_2_371_WI_3,2016-01-26,0,HOUSEHOLD_2_371,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,1824
58033,58033,HOUSEHOLD_2_371_WI_3,2016-01-27,0,HOUSEHOLD_2_371,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,1825
58034,58034,HOUSEHOLD_2_371_WI_3,2016-01-28,2,HOUSEHOLD_2_371,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,1826


In [ ]:
S_base.head()

,Unnamed: 0,unique_id,model,ds,mu_hat_H,SS_baseline,S_baseline,y_true_H
0,0,FOODS_1_145_CA_1,AutoARIMA,2015-01-31,5.240741,4.229612,10,4.0
1,1,FOODS_1_145_CA_1,AutoARIMA,2015-02-01,5.240741,4.229612,10,3.0
2,2,FOODS_1_145_CA_1,AutoARIMA,2015-02-02,5.240741,4.229612,10,2.0
3,3,FOODS_1_145_CA_1,AutoARIMA,2015-02-03,5.318847,4.229612,10,3.0
4,4,FOODS_1_145_CA_1,AutoARIMA,2015-02-04,5.396953,4.229612,10,2.0


In [ ]:
import numpy as np
import pandas as pd

# Simulation function
def simulate_RS_basestock(
    demand_series: pd.Series,
    target_series: pd.Series,
    init_onhand: float,
    lead_time: int,
    review_period: int = 1,
    label: str = "",
    model_name: str | None = None,
):
    if (not label) and model_name:
        label = str(model_name)

    if len(demand_series) != len(target_series):
        raise ValueError("Demand and target length mismatch.")
    if lead_time < 0 or review_period <= 0:
        raise ValueError("lead_time>=0 and review_period>=1 required.")

    d = demand_series.reset_index(drop=True).astype(float).to_numpy()
    s = target_series.reset_index(drop=True).astype(float).to_numpy()
    if np.isnan(d).any() or np.isnan(s).any():
        raise ValueError("Series contain NaNs.")

    N = len(d)
    start_inv  = np.zeros(N, dtype=float)
    receipt    = np.zeros(N, dtype=float)
    end_inv    = np.zeros(N, dtype=float)
    order_qty  = np.zeros(N, dtype=float)

    start_inv[0] = float(init_onhand)

    for t in range(N):
        if t > 0:
            start_inv[t] = end_inv[t-1]
        end_inv[t] = start_inv[t] + receipt[t] - d[t]

        # in trainst inventory
        in_transit = receipt[t+1 : min(t+lead_time, N)].sum() if lead_time >= 2 else 0.0

        # Periodic inspection：t = 0, R, 2R, ...
        if (t % review_period) == 0:
            ip = end_inv[t] + in_transit  # inventory position
            if ip < s[t]:
                qty = s[t] - ip
                order_qty[t] = qty
                if t + lead_time < N:
                    receipt[t + lead_time] += qty

    # KPI
    backlog_mask  = end_inv < 0
    backlog_days  = int(backlog_mask.sum())
    backlog_units = float(end_inv[backlog_mask].sum())
    total_demand  = float(d.sum())
    total_orders  = float(order_qty.sum())
    order_count   = int((order_qty > 0).sum())

    pos_onhand_sum = float(end_inv[end_inv > 0].sum())
    avg_onhand     = pos_onhand_sum / N if N else 0.0
    avg_demand     = float(d.mean()) if N else 0.0
    turnover_daily = (avg_demand / avg_onhand) if avg_onhand > 1e-12 else 0.0

    fill_rate = (total_demand + backlog_units) / total_demand if total_demand > 1e-12 else 1.0
    fill_rate = max(0.0, min(1.0, fill_rate))
    psl = 1.0 - (backlog_days / N) if N else 1.0

    # CSL
    csl_ok = csl_total = 0
    if lead_time > 0:
        for t in range(N):
            if order_qty[t] > 0 and (t + lead_time - 1) < N:
                if not (end_inv[t : t + lead_time] < 0).any():
                    csl_ok += 1
                csl_total += 1
    csl = (csl_ok / csl_total) if csl_total > 0 else float("nan")

    return {
        "fill_rate":       fill_rate,
        "psl":             psl,
        "csl":             csl,
        "turnover_daily":  turnover_daily,
        "order_count":     order_count,
        "avg_order_qty":   (total_orders / order_count) if order_count else 0.0,
    }


In [ ]:
# Batch run: Adapt to S table & demand table of 159 items
def run_rs_for_all(
    S_df: pd.DataFrame,
    D_df: pd.DataFrame,
    lead_time: int = 2,
    review_period: int = 1,
    init_policy: str | float = 'S0',
    s_col: str = 'S',
):
    S = S_df.copy()
    D = D_df.copy()

    # S：unique_id / model / ds / S
    need_S = {'unique_id', 'model', 'ds', s_col}
    miss_S = need_S - set(S.columns)
    if miss_S:
        raise ValueError(f"S_df missing: {miss_S}")

    # Demand：unique_id / ds(or date) / sales
    if 'ds' not in D.columns and 'date' in D.columns:
        D = D.rename(columns={'date': 'ds'})
    if 'demand' not in D.columns and 'sales' in D.columns:
        D = D.rename(columns={'sales': 'demand'})

    need_D = {'unique_id', 'ds', 'demand'}
    miss_D = need_D - set(D.columns)
    if miss_D:
        raise ValueError(f"D_df missing: {miss_D}")

    if not np.issubdtype(S['ds'].dtype, np.datetime64):
        S['ds'] = pd.to_datetime(S['ds'])
    if not np.issubdtype(D['ds'].dtype, np.datetime64):
        D['ds'] = pd.to_datetime(D['ds'])

    # Run on (unique_id, model)
    kpi_rows = []
    for (uid, mdl), gS in S.groupby(['unique_id', 'model'], sort=False):
        gS = gS.sort_values('ds')[['ds', s_col]].rename(columns={s_col: 'S_target'})

        gD = (D[D['unique_id'] == uid][['ds', 'demand']]
                .merge(gS[['ds']], on='ds', how='right')
                .fillna({'demand': 0.0})
                .sort_values('ds'))

        S_series = gS['S_target'].astype(float).reset_index(drop=True)
        D_series = gD['demand'].astype(float).reset_index(drop=True)

        init_onhand = float(S_series.iloc[0]) if init_policy == 'S0' else float(init_policy)

        res = simulate_RS_basestock(
            demand_series = D_series,
            target_series = S_series,
            init_onhand   = init_onhand,
            lead_time     = lead_time,
            review_period = review_period,
            model_name    = f"{uid} | {mdl}",
        )

        kpi_rows.append({
            'unique_id'      : uid,
            'model'          : mdl,
            'n_days'         : int(len(D_series)),
            'orders'         : res['order_count'],
            'avg_order_qty'  : res['avg_order_qty'],
            'fill_rate'      : res['fill_rate'],
            'PSL'            : res['psl'],
            'CSL'            : res['csl'],
            'turnover_daily' : res['turnover_daily'],
        })

    kpi = pd.DataFrame(kpi_rows).sort_values(['unique_id','model']).reset_index(drop=True)
    return kpi

In [ ]:
# Rename the 'S_baseline' column to 'S' in S_base
S_base = S_base.rename(columns={'S_baseline': 'S'})

kpi_ml_159 = run_rs_for_all(
    S_df          = S_base,
    D_df          = demand_1y,
    lead_time     = 2,
    review_period = 1,
    init_policy   = 'S0',            # Take S as the initial inventory on the first day
    s_col         = 'S'
)

display(kpi_ml_159.head())
print("TOTAL rows:", kpi_ml_159.shape[0])

,unique_id,model,n_days,orders,avg_order_qty,fill_rate,PSL,CSL,turnover_daily
0,FOODS_1_145_CA_1,AutoARIMA,361,252,2.638889,0.986446,0.986150,0.964143,0.246657
1,FOODS_1_145_CA_1,CrostonClassic,361,249,2.674699,0.986446,0.983380,0.955645,0.241630
2,FOODS_1_145_CA_1,SeasonalNaive,361,185,3.578378,0.993976,0.991690,0.967568,0.175290
3,FOODS_1_145_CA_1,Theta,361,248,2.681452,0.984940,0.980609,0.947368,0.242868
4,FOODS_1_145_CA_3,AutoARIMA,361,243,2.629630,0.984399,0.988920,0.966942,0.213028


TOTAL rows: 636


In [ ]:
kpi_baseline_159 = kpi_ml_159.copy()

In [ ]:
kpi_baseline_159.to_csv("/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/kpi_baseline_159.csv", index=False)